In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

%matplotlib inline

In [2]:
july_trip = pd.read_csv('../data/july_trip.csv')

In [3]:
july_trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134926 entries, 0 to 134925
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   pubTimeStamp    134926 non-null  object 
 1   companyName     134926 non-null  object 
 2   tripRecordNum   134926 non-null  object 
 3   sumdID          134926 non-null  object 
 4   tripDuration    134926 non-null  float64
 5   tripDistance    134926 non-null  float64
 6   startDate       134926 non-null  object 
 7   startTime       134926 non-null  object 
 8   endDate         134926 non-null  object 
 9   endTime         134926 non-null  object 
 10  startLatitude   134926 non-null  float64
 11  startLongitude  134926 non-null  float64
 12  endLatitude     134926 non-null  float64
 13  endLongitude    134926 non-null  float64
 14  tripRoute       134926 non-null  object 
 15  create_dt       134926 non-null  object 
dtypes: float64(6), object(10)
memory usage: 16.5+ MB


In [4]:
july_trip['start_dt'] = pd.to_datetime(pd.to_datetime(july_trip['startDate']).astype(str) + ' ' + july_trip['startTime'])
july_trip['end_dt'] = pd.to_datetime(pd.to_datetime(july_trip['endDate']).astype(str) + ' ' + july_trip['endTime'])
july_trip['time_diff'] = (july_trip.end_dt - july_trip.start_dt )
july_trip.head()

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt,time_diff
0,2019-07-01 00:02:52.213000,Bird,BRD5179,PoweredZSHKJ,2.0,0.0,2019-07-01 00:00:00,00:01:32.290000,2019-07-01 00:00:00,00:03:55.966666,36.1775,-86.7516,36.1778,-86.7514,"[[36.177438,-86.751861],[36.177425,-86.751987]...",2019-07-02 05:30:18.333000,2019-07-01 00:01:32.290000,2019-07-01 00:03:55.966666,0 days 00:02:23.676666
1,2019-07-01 00:02:52.213000,Bird,BRD5177,Powered22JF6,0.0,0.0,2019-07-01 00:00:00,00:02:38.640000,2019-07-01 00:00:00,00:02:54.406666,36.1627,-86.7711,36.1627,-86.7711,"[[36.162718,-86.771102],[36.162718,-86.771102]]",2019-07-02 05:30:18.273000,2019-07-01 00:02:38.640000,2019-07-01 00:02:54.406666,0 days 00:00:15.766666
2,2019-07-01 00:02:52.213000,Bird,BRD5176,Powered22JF6,1.0,0.0,2019-07-01 00:00:00,00:03:04.333333,2019-07-01 00:00:00,00:03:37.680000,36.1627,-86.7713,36.1627,-86.7713,"[[36.162734,-86.77122],[36.162706,-86.771325],...",2019-07-02 05:30:18.243000,2019-07-01 00:03:04.333333,2019-07-01 00:03:37.680000,0 days 00:00:33.346667
3,2019-07-01 00:02:52.213000,Bird,BRD5178,PoweredIB2CC,1.0,0.0,2019-07-01 00:00:00,00:02:06.963333,2019-07-01 00:00:00,00:03:25.766666,36.1525,-86.7988,36.1525,-86.7988,"[[36.152499,-86.798708]]",2019-07-02 05:30:18.303000,2019-07-01 00:02:06.963333,2019-07-01 00:03:25.766666,0 days 00:01:18.803333
4,2019-07-01 00:02:52.213000,Bird,BRD5180,PoweredW4G7R,2.0,0.0,2019-07-01 00:00:00,00:01:19.843333,2019-07-01 00:00:00,00:02:53.976666,36.1578,-86.7760,36.1575,-86.7758,"[[36.157523,-86.775794]]",2019-07-02 05:30:18.363000,2019-07-01 00:01:19.843333,2019-07-01 00:02:53.976666,0 days 00:01:34.133333


In [5]:
july_trip.companyName.value_counts()

Bird             44235
Lime             38397
Lyft             28899
SPIN             11685
Bolt Mobility     6759
JUMP              4089
Gotcha             862
Name: companyName, dtype: int64

In [6]:
lime = july_trip.loc[(july_trip['companyName'] == 'Lime')]
lyft = july_trip.loc[(july_trip['companyName'] == 'Lyft')]
bird = july_trip.loc[(july_trip['companyName'] == 'Bird')]
spin = july_trip.loc[(july_trip['companyName'] == 'SPIN')]
bolt = july_trip.loc[(july_trip['companyName'] == 'Bolt Mobility')]
gotcha = july_trip.loc[(july_trip['companyName'] == 'Gotcha')]
jump = july_trip.loc[(july_trip['companyName'] == 'JUMP')]

In [7]:
lime.shape

(38397, 19)

In [8]:
lyft.head()

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt,time_diff
5,2019-07-01 00:03:52.060000,Lyft,LFT7,Powered840423,3.716183,2375.32816,2019-07-01 00:00:00,00:00:08.963333,2019-07-01 00:00:00,00:03:51.933333,36.14933,-86.79760,36.15265,-86.79706,"[[36.14933,-86.7976],[36.14942,-86.7975],[36.1...",2019-07-02 09:34:36.953000,2019-07-01 00:00:08.963333,2019-07-01 00:03:51.933333,0 days 00:03:42.970000
10,2019-07-01 00:08:01.777000,Lyft,LFT11,Powered414680,6.297550,5488.84532,2019-07-01 00:00:00,00:01:43.776666,2019-07-01 00:00:00,00:08:01.630000,36.15249,-86.79472,36.13892,-86.80048,"[[36.15249,-86.79472],[36.15235,-86.79489],[36...",2019-07-02 09:34:37.087000,2019-07-01 00:01:43.776666,2019-07-01 00:08:01.630000,0 days 00:06:17.853334
11,2019-07-01 00:08:15.163000,Lyft,LFT12,Powered635347,6.776617,5383.85844,2019-07-01 00:00:00,00:01:28.440000,2019-07-01 00:00:00,00:08:15.036666,36.15234,-86.79483,36.13895,-86.80047,"[[36.15234,-86.79483],[36.15219,-86.79494],[36...",2019-07-02 09:34:37.120000,2019-07-01 00:01:28.440000,2019-07-01 00:08:15.036666,0 days 00:06:46.596666
12,2019-07-01 00:09:50.063000,Lyft,LFT15,Powered883367,7.443667,4379.92140,2019-07-01 00:00:00,00:02:23.333333,2019-07-01 00:00:00,00:09:49.953333,36.13676,-86.80596,36.13664,-86.80535,"[[36.13676,-86.80596],[36.13695,-86.80611],[36...",2019-07-02 09:34:37.217000,2019-07-01 00:02:23.333333,2019-07-01 00:09:49.953333,0 days 00:07:26.620000
13,2019-07-01 00:10:03.083000,Lyft,LFT16,Powered937680,1.487350,104.98688,2019-07-01 00:00:00,00:08:33.696666,2019-07-01 00:00:00,00:10:02.936666,36.14824,-86.80669,36.14834,-86.80684,"[[36.14824,-86.80669],[36.1483,-86.80679],[36....",2019-07-02 09:34:37.253000,2019-07-01 00:08:33.696666,2019-07-01 00:10:02.936666,0 days 00:01:29.240000


In [9]:
jump.head(10)

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt,time_diff
2994,2019-07-01 19:58:07,JUMP,JMP64,Powered7038f9f7-7285-5a3d-8635-df97d6c53c73,21.434584,12249.6,2019-07-01 00:00:00,15:08:58,2019-07-01 00:00:00,15:30:24,36.152416,-86.795006,36.152843,-86.793150,"[[""36.152333"",""-86.7951""],[""36.152343"",""-86.79...",2019-07-02 07:58:42.137000,2019-07-01 15:08:58,2019-07-01 15:30:24,0 days 00:21:26
2995,2019-07-01 19:58:07,JUMP,JMP68,Powered0e871aa8-de0a-5b5b-9909-333de5502aa0,21.616800,9979.2,2019-07-01 00:00:00,15:27:46,2019-07-01 00:00:00,15:49:23,36.151066,-86.804146,36.160225,-86.778080,"[[""36.151125"",""-86.804055""],[""36.151103"",""-86....",2019-07-02 07:58:42.787000,2019-07-01 15:27:46,2019-07-01 15:49:23,0 days 00:21:37
2996,2019-07-01 19:58:07,JUMP,JMP55,Powered3d099838-659b-5660-9463-a029ba277d63,828.387000,8236.8,2019-07-01 00:00:00,00:15:52,2019-07-01 00:00:00,14:04:15,36.164593,-86.776955,36.165330,-86.800125,"[[""36.164603"",""-86.776953""],[""36.164616"",""-86....",2019-07-02 07:58:39.650000,2019-07-01 00:15:52,2019-07-01 14:04:15,0 days 13:48:23
2997,2019-07-01 19:58:07,JUMP,JMP63,Powered0a80f289-6ad7-5354-ab58-0bf7bdaf8a1a,10.221517,4752.0,2019-07-01 00:00:00,15:13:55,2019-07-01 00:00:00,15:24:08,36.150417,-86.800940,36.137707,-86.800710,"[[""36.150418"",""-86.80094""],[""36.150465"",""-86.8...",2019-07-02 07:58:41.847000,2019-07-01 15:13:55,2019-07-01 15:24:08,0 days 00:10:13
2998,2019-07-01 19:58:07,JUMP,JMP62,Powerede90c6861-15dd-5072-bd69-60aad4262311,10.497383,4752.0,2019-07-01 00:00:00,15:13:22,2019-07-01 00:00:00,15:23:52,36.150482,-86.801000,36.137794,-86.800835,"[[""36.150418"",""-86.800951""],[""36.150418"",""-86....",2019-07-02 07:58:41.810000,2019-07-01 15:13:22,2019-07-01 15:23:52,0 days 00:10:30
2999,2019-07-01 19:58:07,JUMP,JMP66,Poweredab6713c3-7459-53c2-9c45-e0112f720aad,52.033050,20011.2,2019-07-01 00:00:00,14:48:07,2019-07-01 00:00:00,15:40:09,36.164780,-86.779660,36.164040,-86.779260,"[[""36.164778"",""-86.779665""],[""36.164746"",""-86....",2019-07-02 07:58:42.357000,2019-07-01 14:48:07,2019-07-01 15:40:09,0 days 00:52:02
3000,2019-07-01 19:58:07,JUMP,JMP57,Powered70e93b4e-0895-59ac-b16d-1b53f04c1dc3,10.586567,5016.0,2019-07-01 00:00:00,14:01:46,2019-07-01 00:00:00,14:12:21,36.169310,-86.802704,36.169130,-86.802580,"[[""36.169293"",""-86.802611""],[""36.169293"",""-86....",2019-07-02 07:58:41.017000,2019-07-01 14:01:46,2019-07-01 14:12:21,0 days 00:10:35
3001,2019-07-01 19:58:07,JUMP,JMP59,Powered1596b61a-225f-5a94-9027-cfc92d1075a0,56.271915,0.0,2019-07-01 00:00:00,13:25:43,2019-07-01 00:00:00,14:21:59,36.168510,-86.771805,36.168510,-86.771805,"[[""36.168511"",""-86.771805""],[""36.168511"",""-86....",2019-07-02 07:58:41.360000,2019-07-01 13:25:43,2019-07-01 14:21:59,0 days 00:56:16
3002,2019-07-01 19:58:07,JUMP,JMP65,Poweredbb60ef04-c719-597f-805f-20978964fd8c,51.504883,19272.0,2019-07-01 00:00:00,14:48:21,2019-07-01 00:00:00,15:39:51,36.164806,-86.779884,36.163986,-86.778360,"[[""36.16467"",""-86.779981""],[""36.16467"",""-86.77...",2019-07-02 07:58:42.290000,2019-07-01 14:48:21,2019-07-01 15:39:51,0 days 00:51:30
3003,2019-07-01 19:58:07,JUMP,JMP67,Powered6e8b5829-e580-5a6f-b26a-10f9d586b75b,53.130066,20486.4,2019-07-01 00:00:00,14:47:06,2019-07-01 00:00:00,15:40:14,36.164684,-86.779760,36.164093,-86.779330,"[[""36.164686"",""-86.779766""],[""36.16484"",""-86.7...",2019-07-02 07:58:42.323000,2019-07-01 14:47:06,2019-07-01 15:40:14,0 days 00:53:08


In [14]:
lime_time_1 = lime.time_diff/pd.Timedelta(hours=1)
lime_time_1.sum()

9636.745277777778

In [15]:
lime_5 = lime.time_diff/pd.Timedelta(minutes=1)
round(lime_5.mean(),3)

15.059

In [16]:
lyft_5 = lyft.time_diff/pd.Timedelta(minutes=1)
round(lyft_5.mean(),3)

15.884

In [17]:
bird_5 = bird.time_diff/pd.Timedelta(minutes=1)
round(bird_5.mean(),3)

13.302

In [18]:
spin_5 = spin.time_diff/pd.Timedelta(minutes=1)
round(spin_5.mean(),3)

20.687

In [19]:
bolt_5 = bolt.time_diff/pd.Timedelta(minutes=1)
round(bolt_5.mean(),3)

27.403

In [20]:
gotcha_5 = gotcha.time_diff/pd.Timedelta(minutes=1)
round(gotcha_5.mean(),3)

9.303

In [21]:
jump_5 = jump.time_diff/pd.Timedelta(minutes=1)
round(jump_5.mean(),3)

26.252